In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Let's get this out of the way once and for all: Clickbaiting is the deliberate act of over-promising or otherwise misrepresenting what you'll discover when you read a storey on the internet – in a headline, on social media, in an image, or in some combination.
We'll try to figure out which title attributes of the YouTube channel '5 minutes craft' influence the number of views, and then develop a model that predicts the number of views (per day to standardise) only on the basis of the title. 

In [2]:
df= pd.read_csv('../input/5minute-crafts-video-views-dataset/5-Minute Crafts.csv')
df.head()

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=600
fig=plt.figure(figsize=(2,0.1), facecolor= '#f6f5f5')
gs= fig.add_gridspec(1,1)
gs.update(wspace=0, hspace=0)

background_color= "#f6f5f5"

ax0= fig.add_subplot(gs[0,0])
ax0.set_facecolor(background_color)
for s in ['top', 'left', 'right', 'bottom']:
    ax0.spines[s].set_visible(False)

ax0.set_xticks([])
ax0.set_yticks([])
ax0.grid(which='major', axis='y', zorder=0, color='#EEEEEE')
ax0.text(0, 0.79, '4978', color='#A8F712', fontsize=20, ha='center', weight='bold', va='bottom')
ax0.text(0, 0, 'Number of rows', color='#bbd092', fontsize=6, ha='center', va='top', weight='bold')
ax0.text(0.9, 0.79, '15', color='#A8F712', fontsize=20, ha='center', weight='bold', va='bottom')
ax0.text(0.9, 0, 'Number of columns', color='#bbd092', fontsize=6, ha='center', va='top', weight='bold')


In [4]:
display(df.columns)
display(df.index)

In [5]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
df1= df[df['total_views']<100000000].copy()
x=df1['num_chars']
y=df1['total_views']
data= [go.Scatter(x=x, 
                   y=y, 
                   mode = 'markers',
                   marker=dict(
                             
                               color='#bbd092',
                               line = {'width':1}))]
layout= go.Layout(title= 'Total number of Views vs Character used in the Title', xaxis= {'title': 'Number of character used'},
                 yaxis={'title': 'Total Number of views'})
fig= go.Figure(data=data, layout=layout)
fig.show()

In [6]:
fig=go.Figure()
data= go.Histogram(x=df.title_sentiment,nbinsx=15, histnorm='percent' ,
    marker_color='#A8F712')
fig.add_trace(data)
fig.update_layout(
    title_text='Sentiment Distribution', # title of plot
    xaxis_title_text='Title Sentiment', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [7]:
fig=go.Figure()
data= go.Histogram(x=df.avg_word_len,nbinsx=15, histnorm='percent' ,
    marker_color='#bbd092')
fig.add_trace(data)
fig.update_layout(
    title_text='Average Word Length Distribution', # title of plot
    xaxis_title_text='Average Word Length', # xaxis label
    yaxis_title_text='Percentage', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [8]:
import seaborn as sns
vc = df[df['total_views'] < 1000000].groupby("num_words").mean().reset_index()
plt.figure(figsize=(12,6))
palette= sns.color_palette("crest")
ax = sns.barplot(x="num_words", y="total_views", data=vc, palette= palette)

plt.title("Average Views by Number of Words Used");
plt.xlabel("Number of Words Used", fontsize=14);
plt.ylabel("Average Views", fontsize=14);

In [9]:
corr = df.corr()
fig,ax=plt.subplots(figsize=(15,15))
mask = np.zeros_like(corr, dtype='bool')
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask=mask, annot=True, cmap=sns.color_palette("crest", as_cmap=True), square=True, ax=ax, vmin = -1.0, vmax = 1.0, linewidths=.5)


In [10]:
df['views_per_day'] = df['total_views'] / df['active_since_days']
ax0_sns = sns.histplot(x = df['views_per_day'], zorder = 3, linewidth = 0.6, bins = 25, log_scale = True, color = "#A8F712")

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
dataset=df.drop(columns=['video_id', 'title'])
X = dataset.drop(columns = ['views_per_day'])
y = dataset['views_per_day']


choices = [1, 0]
for col in X.columns:
    scaler = StandardScaler()
    X[col] = scaler.fit_transform(pd.DataFrame(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [13]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [14]:
from sklearn.metrics import mean_squared_error, r2_score
def def_r2_score(y_test, y_pred):
    r2s = r2_score(y_test, y_pred)
    print("r2s of testing model", r2s)

def def_mean_squared_error(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    print('mse of testing model', mse)
    rmse = mse**(1/2)
    print('rmse of testing model', rmse)


import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
train_results1 = model.predict(X_train)


def_r2_score(y_test, y_pred)
def_mean_squared_error(y_test, y_pred)
mse2 = mean_squared_error(y_train, train_results1)
print('rmse of training model', mse2**0.5)

In [15]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred_rf= rf.predict(X_test)

#Model 2: Random forest Regression.
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)

y_pred_rf= rf.predict(X_test)
train_results2 = model.predict(X_train)

def_r2_score(y_test, y_pred_rf)
def_mean_squared_error(y_test, y_pred_rf)

mse3 = mean_squared_error(y_train, train_results2)
print('rmse of training model', mse3**0.5)

In [16]:
from sklearn.model_selection import GridSearchCV
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model,parameters, cv=None)
grid.fit(X_train, y_train)
print ("r2 / variance : ", grid.best_score_)
print("Residual sum of squares: %.2f"
              % np.mean((grid.predict(X_test) - y_test) ** 2))
y_pred_grid= grid.predict(X_test)

def_r2_score(y_test, y_pred_grid)
def_mean_squared_error(y_test, y_pred_grid)
